In [69]:
import numpy as np
import matplotlib.pyplot as plt
from random import seed
import random
from copy import deepcopy
import timeit
from pandas import DataFrame

## Finding Delivery route for a given Pickup route and stack loading

Delivery route is constructed by choosing in a greedy way (whichever is closer) to the current point. 
At any time, we will have three options from three stacks to choose from. Whichever of the three is closest to the current location will be chosen and then process will be continued until we have visited all the nodes.   

In [25]:
# finding delivery route in a greedy way

def delivery(p, s):
    
    d = np.zeros(np.shape(p))
    d[0] = p[0]  # depot is same for both

    m, stack = np.shape(s)

    k = 0 
    q = stack-1
    ks = np.array([q, q, q])  
    a = np.array([1, 1, 1])

    def dp(X1, X2):
        
        # returns distance between two locations
        x1, y1 = X1
        x2, y2 = X2
        z  = np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
        return z

    z = np.zeros(m)
    n = len(p)  # 34 in 1st case
    
    while(k < n-1):
        
        # finding the closest point out of three available options
        
        if (ks[0] >= 0 and ks[1] >= 0 and ks[2] >= 0):
            a[0] = int(np.where(p[:, 0] == s[0, ks[0]])[0])
            a[1] = int(np.where(p[:, 0] == s[1, ks[1]])[0])
            a[2] = int(np.where(p[:, 0] == s[2, ks[2]])[0])
            z = dp(p[a[0]][1:], d[k][1:]), dp(p[a[1]][1:], d[k][1:]), dp(p[a[2]][1:], d[k][1:])
            k = k + 1

            d[k] = p[a[np.argmin(z)]]            
            ks[np.argmin(z)] = ks[np.argmin(z)] - 1


        elif(ks[0] >= 0 and ks[1] >= 0 and ks[2] < 0 ):            
            a[0] = int(np.where(p[:, 0] == s[0, ks[0]])[0])
            a[1] = int(np.where(p[:, 0] == s[1, ks[1]])[0])
            z = dp(p[a[0]][1:], d[k][1:]), dp(p[a[1]][1:], d[k][1:]), 1000
            k = k + 1
            
            d[k] = p[a[np.argmin(z)]]             
            ks[np.argmin(z)] = ks[np.argmin(z)] - 1

        elif(ks[0] >= 0 and ks[1] < 0 and ks[2] >= 0 ):            
            a[0] = int(np.where(p[:, 0] == s[0, ks[0]])[0])
            a[2] = int(np.where(p[:, 0] == s[2, ks[2]])[0])
            z = dp(p[a[0]][1:], d[k][1:]), 1000, dp(p[a[2]][1:], d[k][1:])
            k = k + 1
            
            d[k] = p[a[np.argmin(z)]]
            ks[np.argmin(z)] = ks[np.argmin(z)] - 1

        elif(ks[0] < 0 and ks[1] >= 0 and ks[2] >= 0 ):
            a[1] = int(np.where(p[:, 0] == s[1, ks[1]])[0])
            a[2] = int(np.where(p[:, 0] == s[2, ks[2]])[0])

            z = 1000, dp(p[a[1]][1:], d[k][1:]), dp(p[a[2]][1:], d[k][1:])
            k = k + 1
            
            d[k] = p[a[np.argmin(z)]]
            ks[np.argmin(z)] = ks[np.argmin(z)] - 1

        elif(ks[0] < 0 and ks[1] < 0 and ks[2] >= 0):
            a[2] = int(np.where(p[:, 0] == s[2, ks[2]])[0])
            z = 1000, 1000, dp(p[a[1]][1:], d[k][1:])
            k = k + 1
            
            d[k] = p[a[np.argmin(z)]]
            ks[2] = ks[2] - 1

        elif(ks[0] < 0 and ks[1] >= 0 and ks[2] < 0):
            a[1] = int(np.where(p[:, 0] == s[1, ks[1]])[0])
            z = 1000, dp(p[a[1]][1:], d[k][1:]), 1000
            k = k + 1
            
            d[k] = p[a[np.argmin(z)]]
            ks[1] = ks[1] - 1

        elif(ks[0] >= 0 and ks[1] < 0 and ks[2] < 0):
            a[0] = int(np.where(p[:, 0] == s[0, ks[0]])[0])
            z = dp(p[a[1]][1:], d[k][1:]), 1000, 1000
            k = k + 1
            
            d[k] = p[a[np.argmin(z)]]
            ks[0] = ks[0] - 1
            
    return d


## Generate Initial solution for given pickup and delivery locations using randomized generation method. 

A pickup route is generated by randomly shuffling all the order locations. Stack loading is then determined by assigning all the orders from pickup to the same stack until it is full and then the next stack is considered. For delivery route, the method defined above is used. 

In [26]:
# Generate Initial Solution for a given pickup and delivery route using randomized generation method

def initial_sol(p0, d0, seed):
    
    m = 3
    n = len(d0)-1
    p = deepcopy(p0)
    d = deepcopy(d0)
    
    p1 = p[1:, :]  # shuffling every other point except depot
    np.random.seed(seed)
    np.random.shuffle(p1)  #pickup route generation by randomization

    k = 0 
    if (n%m == 0):
        stack = int(n/m)
    else:
        stack = int(n/m) + 1

    s = np.zeros([m, stack])

    # stack construction -
    for i in range(m):
        for j in range(stack):
            s[i][j] = int(p1[k,0])
            k = k+1

    # delivery route construction
    d2 = delivery(p, s)
#     d1 = p1[::-1]
#     d2 = np.vstack((d[0, :], d1))
    
    S = np.array([p, d2, s])
    
    return S

## Calculating the cost/distance of a given route 

Euclidian distance is considered to calculate the cost of travelling between two points. The length of the whole path is thus the sum of all distances between two order visited consecutively in the route. 

In [27]:
# calculate cost/distance of a given route

def dist(route):
    
    ed = 0  #euclidean distance

    for i in range(len(route)-1):
        dx = route[i+1][1] - route[i][1]
        dy = route[i+1][2] - route[i][2]
        ed = ed + np.sqrt(dx**2 + dy**2)
        
    return ed

def distance(S):
    return dist(S[0]) + dist(S[1])

## Neighborhood Structures

## 1. Route-Swap 

The neighborhood of the initial solution is obtained by swapping two consecutive locations in the pickup route. Accordingly, stack and delivery are modified to keep the solution feasible. We keep doing this for all the possible locations (2(n-1) in this case) and keep a track of the best solution i.e. the one with the minimum distance. 



In [28]:
# route-swap algorithm

def RS(S1, i ,j):
    
    """ Interchange two consecutive orders
        i - 1st order
        j - next order of i """
    
    S = deepcopy(S1)
    r = S[0]
    
    r1 = r[1:, :]
    temp1 = deepcopy(r1[i])
    r1[i] = r1[i-1 +2*j]
    r1[i-1+2*j] = temp1 
   
    k = 0
    n = len(r)-1
    m = 3
    
    if (n%m == 0):
        stack = int(n/m)
    else:
        stack = int(n/m) + 1
    
    st2 = np.zeros([m, stack])
    
    for i in range(m):
        for j in range(int(n/m)):
            st2[i][j] = int(r1[k,0])
            k = k+1
     
    
    # change position in delivery accordingly
    d3 = delivery(r, st2)
    
#     d2 = r1[::-1]
#     d3 = np.vstack((S[1][0, :], d2))
    #d2 = S[1][0,:] + d2
    
    S = np.array([r, d3, st2])
    
    return S

def min_RS(S):
    
    """ find the minimum distance by trying all possible combination"""
    dst = dist(S[0]) + dist(S[1]) 
    S2 = deepcopy(S)
    n = len(S[0])-1 
    
    for i in range(1, n):
        for j in range(0, 2):
            
            if(i==n-1):
                j = 0
                
            S1 = RS(S, i, j)
            
            dst1 = dist(S1[0]) + dist(S1[1])
            
            if (dst1 < dst):
                dst = dst1
                S2 = deepcopy(S1)
            
    return S2

## 2. Complete-Swap

The neighborhood is obtained by swapping the stack positions of two orders that are assigned to different stacks. Accordingly, pickup and delivery needs to be modified to keep the feasibility of solution.  

In [29]:
# complete swap algorithm 

def CS(S1, i1, j1, i2, j2):
    
    """performs swap in stack"""
    
    S = deepcopy(S1)
    m, q = S[2].shape
    
    #modify stack
    st2 = (S[2])
    
    temp1 = deepcopy(st2[i1][j1])
    st2[i1][j1] = st2[i2][j2]
    st2[i2][j2] = temp1
    
    
    index1 = q*i1 + j1 + 1
    index2 = q*i2 + j2 + 1
    
    # accordingly modify pickup
    r2 = (S[0])
    
    temp2 = deepcopy(r2[index1])
    r2[index1] = r2[index2]
    r2[index2] = temp2
    
    # delivery construction from pickup and stack
    d3 = delivery(r2, st2)
    
#     r1 = r2[1:, :]
    
#     d2 = r1[::-1]
#     d3 = np.vstack((S[1][0, :], d2))
    
    S = np.array([r2, d3, st2])
    
    return S

def min_CS(S, z):
    
    """given a solution, find the minimum """
    # z is number of iterations
    
    S2 = deepcopy(S)
    m, q = S[2].shape
    dst = dist(S[0]) + dist(S[1])  

    random.seed(1)
    for i in range(1, z):
        
        i1 = random.randint(0, m-1)
        j1 = random.randint(0, q-1)
        i2 = random.randint(0, m-1)
        j2 = random.randint(0, q-1)
        
        S1 = CS(S, i1, j1, i2, j2)
        dst1 = dist(S1[0]) + dist(S1[1])
        if (dst1 < dst):
            dst = dst1
            S2 = deepcopy(S1)
    
    return S2

## 3. In-Stack Swap

The neighborhood is obtained by swapping the positions of two orders that are assigned to the same stacks. Accordingly, pickup and delivery needs to be modified to keep the feasibility of solution. 

In [30]:
# In-stack swap

def ISS(S1, i, j1, j2):
    
    """performs swap in stack"""
    
    S = deepcopy(S1)
    
    m, q = S[2].shape
    
    #modify stack
    st2 = (S[2])
    
    temp1 = deepcopy(st2[i][j1])
    st2[i][j1] = st2[i][j2]
    st2[i][j2] = temp1
    
    index1 = q*i + j1 + 1
    index2 = q*i + j2 + 1
    
    # accordingly modify pickup
    r2 = (S[0])
    
    temp2 = deepcopy(r2[index1])
    r2[index1] = r2[index2]
    r2[index2] = temp2
    
    # delivery can just be opposite of pickup
    d3 = delivery(r2, st2)
#     r1 = r2[1:, :]
#     d2 = r1[::-1]
#     d3 = np.vstack((S[1][0, :], d2))
    
    S = np.array([r2, d3, st2])
    
    return S

def min_ISS(S):
    
    """given a solution, find the minimum """
    # z is number of iterations
    
    dst = dist(S[0]) + dist(S[1])  
    S2 = deepcopy(S)  
    m, q = S[2].shape
        
    for i in range(0, m):
        for j in range(0, q):
            for k in range(j+1, q):
                S1 = ISS(S2, i, j, k)
                dst1 = dist(S1[0]) + dist(S1[1])
                if (dst1 < dst):
                    dst = dst1
                    S2 = deepcopy(S1)

    return S2


## 4. Re-Insertion

An order is moved from one position to another position in pickup, stack as well as in delivery route.  In this case, we need to decide four things, the order which should be moved, the position in pickup route to which it should be moved, its new position in delivery route and the stack that will receive order. 

In [31]:
#  Reinsertion

def RI(S1, i, i1):
    
    S = deepcopy(S1)
    
    m = 3
    n = len(S[0])-1
    # move order i from i to i1
    r2  = S[0]
    temp = deepcopy(r2[i])
    r2[i] = r2[i1]
    r2[i1] = temp
    r1 = r2[1:, :]

    k = 0
    
    if (n%m == 0):
        stack = int(n/m)
    else:
        stack = int(n/m) + 1
    
    st2 = np.zeros([m, stack])
    
    for i in range(m):
        for j in range(int(n/m)):
            st2[i][j] = int(r1[k,0])
            k = k+1
     
    
    # change position in delivery accordingly
    d3 = delivery(r2, st2)
#     d2 = r1[::-1]
#     d3 = np.vstack((S[1][0, :], d2))
    
    S = np.array([r2, d3, st2])
    
    return S

def min_RI(S, z):
    
    """given a solution, find the minimum """
    # z is number of iterations
    
    dst = dist(S[0]) + dist(S[1]) 
    S2 = deepcopy(S) 
    n = len(S[0])-1
    random.seed(1)
    for i in range(1, z):
        
        i1 = random.randint(1, n)
        j1 = random.randint(1, n)
        
        S1 = RI(S, i1, j1)
        dst1 = dist(S1[0]) + dist(S1[1])
        if (dst1 < dst):
            dst = dst1
            S2 = deepcopy(S1)
    
    return S2

## 5. r-Route Permutation

The neighborhood is obtained by permuting r orders which are visited continuously in one route. Accordingly, stack assignment and delivery route is modified

In [32]:
#r-route permutation
# default r = 3

def RP(S1, i):
    
    S = deepcopy(S1)
    
    u = S[0][i-1:i+3]
    np.random.seed(1)
    np.random.shuffle(u)
    r2 = S[0]
    r1 = r2[1:, :]
    
    k = 0
    m = 3
    n = len(r2)-1
    
    if (n%m == 0):
        stack = int(n/m)
    else:
        stack = int(n/m) + 1
    
    st2 = np.zeros([m, stack])
    
    for i in range(m):
        for j in range(int(n/m)):
            st2[i][j] = int(r1[k,0])
            k = k+1
     
    
    # change position in delivery accordingly
    d3 = delivery(r2, st2)
#     d2 = r1[::-1]
#     d3 = np.vstack((S[1][0, :], d2))
    
    S = np.array([r2, d3, st2])
    
    return S


def min_RP(S, z):
    
    """given a solution, find the minimum """
    # z is number of iterations
    
    dst = dist(S[0]) + dist(S[1]) 
    S2 = deepcopy(S)
    
    random.seed(1)
    for i in range(1, z):
        
        k = random.randint(1,31)
        S1 = RP(S, k)
        dst1 = dist(S1[0]) + dist(S1[1])
        if (dst1 < dst):
            dst = dst1
            S2 = deepcopy(S1)
    
    return S2


In [57]:
# route-swap algorithm

def RST(S1, i ,j):
    
    """ Interchange two consecutive orders
        i - 1st order
        j - next order of i """
    
    S = deepcopy(S1)
    r = S[0]
    
    r1 = r[1:, :]
    temp1 = deepcopy(r1[i])
    r1[i] = r1[i-1 +2*j]
    r1[i-1+2*j] = temp1 
   
    k = 0
    n = len(r)-1
    m = 3
    
    if (n%m == 0):
        stack = int(n/m)
    else:
        stack = int(n/m) + 1
    
    st2 = np.zeros([m, stack])
    
    for i in range(m):
        for j in range(int(n/m)):
            st2[i][j] = int(r1[k,0])
            k = k+1
     
    
    # change position in delivery accordingly
    d3 = delivery(r, st2)
    
#     d2 = r1[::-1]
#     d3 = np.vstack((S[1][0, :], d2))
    #d2 = S[1][0,:] + d2
    
    S = np.array([r, d3, st2])
    
    return S

def min_RST(S):
    
    """ find the minimum distance by trying all possible combination"""
    dst = dist(S[0]) + dist(S[1]) 
    S2 = deepcopy(S)
    S1 = deepcopy(S)
    
    n = len(S[0])-1 
    
    for i in range(1, n):
        for j in range(0, 2):
            
            if(i==n-1):
                j = 0
                
            S1 = RS(S1, i, j)
            
            dst1 = dist(S1[0]) + dist(S1[1])
            
            if (dst1 < dst):
                dst = dst1
                S2 = deepcopy(S1)
            
    return S2

## 6. Stack Permutation

The neighborhood is obtained by permuting r orders which are loaded consecutively in same stack. Accordingly, pickup and delivery route is modified.

In [47]:
# Stack permutation

r = 3

def SP(S1, i, j):
    
    S = deepcopy(S1)
    
    m = 3
    n = len(S[0])-1
    
    u = np.zeros(r)
    ind = np.zeros(r)
    for k in range(r):
            u[k] = S[2][i][j + k]               
    
    for k in range(r):
        ind[k] = np.where(S[0][:, 0] == u[k])[0][0]

    np.random.seed(1)
    x = np.array([0, 1, 2]) 
    np.random.shuffle(x) 
    
    for kn in range(3):     
        S[0][int(ind[kn])] = S[0][int(ind[x[kn]])]
    
    r1 = S[0][1:, :]
    
    k = 0
    
    if (n%m == 0):
        stack = int(n/m)
    else:
        stack = int(n/m) + 1
    
    st2 = np.zeros([m, stack])
    
    for i in range(m):
        for j in range(int(n/m)):
            st2[i][j] = int(r1[k,0])
            k = k+1
    
    d2 = S[0][::-1]
    d3 = np.vstack((S[1][0, :], d2))

    S2 = np.array([S[0], d3, st2])  
    
    return S2

def min_SP(S, z):
    
    """given a solution, find the minimum by shuffling r orders from same stack"""
    # z is number of iterations
    
    dst = distance(S)  # can improve this by calculating just the change and not the whole distance every time
    S2 = deepcopy(S)
    
    # finding 3 points to be permuted
    random.seed(1)
    n = len(S[0])-1

    for kn in range(1, z):
        
        i = random.randint(0, 2) 
        j = random.randint(1, ((n/m)-2))
        
        S1 = SP(S2, i, j)
        dst1 = distance(S1)
        
        if (dst1 < dst):
            dst = dst1
            S2 = deepcopy(S1)
    
    return S2

Defining a function to call different neighborhood structures for variable negihborhood algorithms

In [58]:
def VN(Sol, k, z = 1000):
    
    """find solution to the kth neighbourhood
       Sol = Initial solution
       z = number of iterations for neighborhood structures with large neighborhood"""
    
    if (k==1):
        Sol = min_RS(Sol)
    elif (k==2):
         Sol = min_CS(Sol, z)
    elif (k==3):
         Sol = min_ISS(Sol)
    elif (k==4):
         Sol = min_RI(Sol, z)
    elif (k==5):
         Sol = min_RP(Sol, z)
    elif (k==6):
         Sol = min_RST(Sol)
        
    return Sol

## Variable Neighborhood Decent (VND)

In this algorithm, a set of neighborhood structures is considered and the algorithm moves from one structure to another when a local optimum is found and starting the search from the first structure every time a better solution is found. The algorithm will converge to a solution when a better solution can not be find in any neighborhood. Thus the final solution is a local optimum with respect to every structure.  

In [59]:
# variable neighbourhood descent

import timeit

start = timeit.default_timer()

def VND(S):
    
    """Moves from one neighbourhood structure to another to find the optimum. 
    Final solution is the local optimum with respect to each neighbourhood structure
    
    S = Initial Solution"""
    
    Sf = deepcopy(S)
    dst0 = distance(Sf)    
    
    improve = True
    
    while(improve):
        
        #Step-1
        k = 1
        
        while(k<7):
            
            #print(k)
            #step-2
            improve  = False

            #step-3
            Sbar = VN(Sf, k)
            dstbar = distance(Sbar)

            #step-4
            if(dstbar < dst0):
                dst0 = dstbar
                Sf = Sbar
                improve = True
                
            #step-5
            if(improve):
                k = 1
            else:
                k = k + 1
            
    return Sf

## General Variable Neighborhood Search 

This algorithm uses VND with an additional perturbation to get out of local optima. Every time a local optimum is found (using VND), perturbation is performed on it to not be trapped to local optimum.  Since it is a non-deterministic algorithm, it is run for a given time or number of iterations.  


In [62]:
#GVNS

## defining dictonary for calling differnt neighborhood structures for Local Search
delta = {1: min_RS, 2: min_CS, 3: min_ISS, 4: min_RI, 5: min_RP, 6: min_SP}

## defining dictonary for calling differnt neighborhood structures for Perturbation
omega = {1: min_RS, 2: min_CS, 3: min_ISS, 4: min_RI}

def perturb(SP, s, k):
        
        """ function to perform random perturbation on solution SP with kth neighborhood
        and s shake operations"""
        
        #step 3.1
        ks = 0  #counter on shake
        Sp = deepcopy(SP)
        
        while(ks < s):
            
            #step-3.2
            if (k==1 or k==3):
                Sp = omega[k](Sp)
            else:
                Sp = omega[k](Sp, 100)
            
            #step-3.3    
            ks = ks + 1    
        return Sp

def GVNS(Sf, itime, s):
    
    """ Given an initial solution, it uses VND to find a solution
    and then performs random shake operations to escape local optimum"""
    
    """Sf = Initial Solution
    itime = computational time limit
    s = number of shake operations"""
    
    #step-1
    S = deepcopy(Sf)
    S_G = deepcopy(S)
    S_L = deepcopy(S)
    
    start = timeit.default_timer()
    kn = 0
    
    A = []
    while( (timeit.default_timer() - start) < itime):
        
        #step-2
        ko = 1
        improve = False
        A.append([(timeit.default_timer() - start), distance(S_G)])
        
        while(ko < 5): 
        
            #step-3
            Scap = perturb(S_L, s, ko)
    
            #step-4
            Sbar = VND(Scap)

            #step-5
            if (distance(Sbar) < distance(S_L)):
                S_L = deepcopy(Sbar)
                improve = True
                
            #step-6    
            if (distance(Sbar) < distance(S_G)):
                S_G = deepcopy(Sbar)
                A.append([(timeit.default_timer() - start), distance(S_G)])
             
            #step-7
            if(improve):
                break
            else:
                ko = ko + 1
            
        kn = kn + 1

    return S_G #, A

## Hybridized Variable Neighborhood Search 

This algorithm two phases, exploration and intensification. In exploration we start with different initial solutions and find multiple good solutions (one corresponding to each initial solution) using GVNS algorithm. The best solution out 
of the initially found solutions is then used in intensification phase to find the final solution.


In [63]:
#Hybridized variable neighbourhood search

def HS(iter, S, itmax, pcost, s, Scap):
    
    """ iter = number of iterations on every solution
    S = initial solution
    itmax = maximum number of consecutive iterations without improving solution
    pcost = maximum allowed deivation from optimal solution
    s = number of shake operations
    Scap = Global optimum solution"""
    
    # Scap = best known solution
    S1 = deepcopy(S)
    S_G = deepcopy(S1)
    
    #step-1
    it = 0
    itm = 0
    
    #step-2
    S2 = VND(S1)    
    if(distance(S2) < distance(S_G)):
        S_G = deepcopy(S2)
    
    while(it < iter):
        
        cost = 100*(distance(S2)/distance(Scap) - 1)
        
        while(itm < itmax and cost < pcost): 
            
            #step-4
            S2 = GVNS(S, 2, s)
            it = it + 1
            
            #step-5
            if (distance(S2) < distance(S_G)):
                S_G = deepcopy(S2)   
            else:
                itm = itm + 1
                
            cost = 100*(distance(S2)/distance(Scap) - 1)
        
        #step-6
        itm = 0
        S2 = deepcopy(S_G)
        
    return S_G


def HVNS(ml, etime, itime, ni, n, pcost, s, itmax, p, d):
    
    """ ml = numbre of initial solutions to be generated
        etime = exploration iteration time
        itime = intensification time
        ni = number of iterations to be performed on initial solution
        n = number of iterations on best solution
        pcost = maximum percentage of cost deviation wrt best known solution
        s = number of random operations performed for perturbation(shake)
        itmax = max number of consecutive iterations without improving solution"""
    
    start = timeit.default_timer()
    A = []
    km = 1  #counter on number of solution   
    
    while ( (timeit.default_timer() - start) < etime):
        
        #print(kn)
        while((timeit.default_timer() - start) < etime and km < ml+1):
           
            #print(km)
            # for fixed seed, lets put km only as seed
            #step-2
            Sinit = initial_sol(p, d, km)
            A.append(distance(Sinit))
            
            #step-3
            if(km==1):
                S_G = Sinit
            #print("3 step")
            
            #step-4    
            S = HS(ni, Sinit, itmax, pcost, s, S_G)
            #print("4 step")
            
            #step-5
            if(distance(S) < distance(S_G)):
                S_G = deepcopy(S) 
                #print("5 step")
            
            #step-6
            km = km + 1
            
        #kn = kn +1
        
    start = timeit.default_timer() 
    #step-7
    while ((timeit.default_timer() - start) < itime):
        S2 = S_G
        S_G = HS(ni, S2, itmax, pcost, s, S_G)
        
    return S_G  #, A


## Generating Solution

Three sets of 20 instances with 33, 66 and 132 orders respectively taken from Peterson and Madsen are used for testing the algorithm. These sets contain location of order for pickup and delivery which were randomly generated in a 100*100 square with depot at (50, 50) and the number of available stack is three. 

All three algorithms VND, GVNS and HVNS are used to generate solution for 33, 66 and 132 orders. The generated solution is then stored in a list and is exported as an excel sheet. 

In [ ]:
## list for storing file names
d = []
p = []

# arrays storing the result for 33 orders
d33_VND = []
d33_GVNS = []
d33_HVNS = []

# arrays storing the result for 66 orders
d66_VND = []
d66_GVNS = []
d66_HVNS = []

# arrays storing the result for 132 orders
d132_VND = []
d132_GVNS = []
d132_HVNS = []

m = 3 # number of avaialable stacks
#s = 3 # shakes for GVNS
seed = 1  

# storing file name
for i in range(10):
    d.append('R0'+str(i)+'d.tsp')
    p.append('R0'+str(i)+'p.tsp')
    
for i in range(10, 20):
    d.append('R'+str(i)+'d.tsp')
    p.append('R'+str(i)+'p.tsp')
    
#kn = 12
kn = np.array([12, 15, 18, 33, 66])
ml = np.array([1, 5, 7, 8, 10])

for j in range(1):
    
    for kr in (kn[1:]):
        print(kr, j)
        a = d[j]
        b = p[j]

        # reading from particular file. 
        ## Note - change the location for running it later if location of data sets is changed
        ## Note - do not change a and b. just change the text before that specifying new location
        a1 = r"C:\Users\swapnila\Desktop\special topics PSE\Project\DTSPMS_DATASETS\dtspms_c132_set0_1\\" + a
        b1 = r"C:\Users\swapnila\Desktop\special topics PSE\Project\DTSPMS_DATASETS\dtspms_c132_set0_1\\" + b
        deli = np.genfromtxt(a1, skip_header = 6 )
        pick = np.genfromtxt(b1, skip_header = 6 )
        
#        S0 = initial_sol(pick[: 13, :], deli[: 13, :], seed)

    #     print(distance(S0))
    #     t = timeit.default_timer() 
    #     S_VND = VND(S0)
    #     print("time:", timeit.default_timer() - t )
    #     print("V:", distance(S_VND))
    #     print(S_VND)

    #     for s in range(5):
    #         S_GVNS, A = GVNS(S0, 100, s)
    #         print("G:", distance(S_GVNS), A)
        
        for m in ml:
            t = timeit.default_timer()
            S_HVNS = HVNS(m, 200, 20, 2, 2, 30, 1, 2, pick[: (kr + 1), :], deli[: (kr+1), :])
            print("H:", distance(S_HVNS), timeit.default_timer() - t)

#     S_HVNS = HVNS(20, 200, 20, 2, 2, 30, 1, 2, pick[: 13, :], deli[: 13, :])
#     print("H:", distance(S_HVNS))

#    d33_VND.append(distance(S_VND))
#    d33_GVNS.append(distance(S_GVNS))
#     d33_HVNS.append(distance(S_HVNS))
    
# kn = 66    
# for j in range(20):
#     #print(kn, j)
#     a = d[j]
#     b = p[j]
#     a1 = r"C:\Users\swapnila\Desktop\special topics PSE\Project\DTSPMS_DATASETS\dtspms_c66_set0_1\\" + a
#     b1 = r"C:\Users\swapnila\Desktop\special topics PSE\Project\DTSPMS_DATASETS\dtspms_c66_set0_1\\" + b
#     deli = np.genfromtxt(a1, skip_header = 6 )
#     pick = np.genfromtxt(b1, skip_header = 6 )
#     S0 = initial_sol(pick, deli, seed)
#     #print(distance(S0))
    
#     S_VND = VND(S0)
#     #print("S:", distance(S_VND))
#     S_GVNS = GVNS(S0, 500, s)
#     #print("G:", distance(S_GVNS))
#     S_HVNS = HVNS(10, 800, 80, 2, 2, 30, 1, 2, pick, deli)
#     #print("H:", distance(S_HVNS))

#     d66_VND.append(distance(S_VND))
#     d66_GVNS.append(distance(S_GVNS))
#     d66_HVNS.append(distance(S_HVNS))

    
# kn = 132
# for j in range(20):
#     #print(kn, j)
#     a = d[j]
#     b = p[j]
#     a1 = r"C:\Users\swapnila\Desktop\special topics PSE\Project\DTSPMS_DATASETS\dtspms_c132_set0_1\\" + a
#     b1 = r"C:\Users\swapnila\Desktop\special topics PSE\Project\DTSPMS_DATASETS\dtspms_c132_set0_1\\" + b
#     deli = np.genfromtxt(a1, skip_header = 6 )
#     pick = np.genfromtxt(b1, skip_header = 6 )
#     S0 = initial_sol(pick, deli, seed)
#     #print(distance(S0))

#     S_VND = VND(S0)
#     S_GVNS = GVNS(S0, 500, s)
#     S_HVNS = HVNS(10, 200, 10, 2, 2, 30, 1, 2, pick, deli)
        
#     d132_VND.append(distance(S_VND))
#     d132_GVNS.append(distance(S_GVNS))
#     d132_HVNS.append(distance(S_HVNS))
    
# Exporting data in an excel file
# best_33 = np.array([1063, 1032, 1065, 1100, 1052, 1008, 1110, 1105, 1109, 1091, 1016, 1001, 1109, 1084, 1034, 1142, 1093, 1073, 1118, 1089])
# df = DataFrame({'Best': best_33, 'VND': d33_VND, 'GVNS': d33_GVNS}) #, 'HVNS': d33_HVNS})
# df.to_excel('Resultshvns.xlsx', sheet_name = '33', index = False)

# best_66 = np.array([1594, 1600, 1576, 1631, 1611, 1528, 1651, 1653, 1607, 1598, 1702, 1575, 1652, 1617, 1611, 1608, 1725, 1627, 1671, 1635])
# df = DataFrame({'Best': best_66, 'VND': d66_VND, 'GVNS': d66_GVNS, 'HVNS': d66_HVNS})
# df.to_excel('Results66.xlsx', sheet_name = '66', index = False)

# best_132 = np.array([2591, 2645, 2639, 2752, 2603, 2616, 2576, 2615, 2638, 2554, 2646, 2632, 2555, 2659, 2605, 2626, 2534, 2569, 2652, 2644])
# df = DataFrame({'Best': best_132, 'VND': d132_VND, 'GVNS': d132_GVNS, 'HVNS': d132_HVNS})
# df.to_excel('Results132.xlsx', sheet_name = '132', index = False)


15 0
H: 804.8018609518672 233.6364049999993
H: 740.0117519983353 235.3369849999981
H: 721.6145197575347 278.8484843999977
H: 721.6145197575347 279.96348039999793
H: 721.6145197575347 278.80215300000054
18 0
H: 843.9547884973736 1158.0976673000005
H: 843.9547884973736 236.5439441999988
H: 843.9547884973736 277.2668210999982
H: 843.9547884973736 318.389192300001
H: 843.9547884973736 250.753994900002
33 0
H: 1474.3043522696917 287.0436683000007
H: 1474.3043522696917 450.46923900000183
H: 1474.3043522696917 52307.274582800004
H: 1474.3043522696917 394.4269604000001
H: 1474.3043522696917 376.76292949999333
66 0
H: 1924.5116988617287 1816.1271911000076


In [56]:
df = DataFrame({'Best': best_33, 'VND': d33_HVNS})#, 'GVNS': d33_GVNS}) #, 'HVNS': d33_HVNS})
df.to_excel('Results18_hvns.xlsx', sheet_name = '33', index = False)


ValueError: arrays must all be same length

In [52]:
S_HVNS

array([array([[ 0.        , 50.        , 50.        ],
       [11.        , 44.14574285, 57.02509775],
       [ 5.        , 20.86909199, 77.88237131],
       [ 8.        , 13.20454698, 94.86322174],
       [15.        ,  5.86959925, 59.09804327],
       [ 2.        ,  0.76357148, 31.21492744],
       [ 4.        , 10.35486722,  2.50464404],
       [ 3.        , 18.52099897,  8.05624181],
       [14.        , 56.58941784, 21.01544615],
       [ 6.        , 70.86608336, 18.60982523],
       [ 1.        , 74.89736342, 16.21127663],
       [10.        , 96.22233152, 72.27823967],
       [12.        , 73.0702383 , 94.6548543 ],
       [13.        , 55.31298798, 90.92865584],
       [ 7.        , 50.80698472, 89.08940036],
       [ 9.        , 46.86839965, 77.34431722]]),
       array([[ 0.        , 50.        , 50.        ],
       [ 9.        , 46.86839965, 77.34431722],
       [ 7.        , 50.80698472, 89.08940036],
       [13.        , 55.31298798, 90.92865584],
       [12.        , 73.